In [7]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [15]:
#Test Code| Version 2#
Year='1935'
Showa='10'
#Read Motherdata
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\"+Year+"\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

In [16]:
#Test code| Version 2#
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

PositionList=['Manager','Leader','Admin','Outsource','Engineer1','Engineer2']

In [30]:
# Function for returning x coordinate of Position name
def Detect_Position(Json,Position):
    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue
            
    ### Get Coordinates ###
    if Position=='Manager':        
        res = [d
           for d in Dict 
           if ('主事' in d['inferText']) or ('主' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
    
    if Position=='Leader':        
        res = [d
           for d in Dict 
           if ('課長' in d['inferText']) or ('課' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Engineer1':        
        res = [d
           for d in Dict 
           if ('技師' in d['inferText']) or ('師' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
    
    if Position=='Engineer2':        
        res = [d
           for d in Dict 
           if ('技手' in d['inferText']) or ('手' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Admin':        
        res = [d
           for d in Dict 
           if ('事務員' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Outsource':
        res = [d
           for d in Dict 
           if '嘱託員' in d['inferText']]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Clerk':
        res = [d
           for d in Dict 
           if d['inferText']=='書記']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            return([])
        
    if Position=='Worker':
        res = [d
           for d in Dict 
           if d['inferText']=='雇']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            return([])
        
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Dept,Office,ImagePage):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"
    with open(path+"Image"+str(ImagePage)+".png",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [67]:
#Test code| Version 2#
for n in range(1,len(df)):
    #Show Working office#
    #Extract key info of office
    Row  = df.iloc[n]
    NxRow= df.iloc[n+1]

    ###Collect Location information###
    Page=int(Row["Page"])
    NxPage=int(NxRow["Page"])

    Dept=Row["Dept"]
    NxDept=NxRow["Dept"]

    Office=Row["Office"]
    NxOffice=NxRow["Office"]
    
    print(Dept,Office)

    #Convert to json via CLOVA
    JsonList=[]
    for l in range(1,len(dta[str(Year)][Dept][Office]['Page_Range'])+1):
        try:
            Json=Clova(Dept,Office,l)                
        except FileNotFoundError:
            break
        JsonList.append(Json)
        Index=0
        for element in JsonList:
            if 'fields' not in list(element.keys()):
                JsonList=JsonList[-0]
            Index=Index+1
                
        
        #Find Starting X coordinates of Positions.
        PositionListNow=PositionList.copy()
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
        Page_Range=dta[str(Year)][Dept][Office]['Page_Range']
        CordDict={}
        Page=Page_Range[0]
        if len(JsonList)==7:
            continue
        else:
            for k in range(0,len(JsonList)):
                CordList=list(map(Detect_Position,[JsonList[k]]*len(PositionListNow),PositionListNow))
                Positions= {PositionListNow[i]:CordList[i] for i in range(len(PositionListNow))}
                Positions = {k:v for k,v in Positions.items() if v}    
                print(Positions)

                PosiList = list(Positions.keys())
                CordList = list(Positions.values())

                CordDictNow= {PosiList[i]: {'XLocation':CordList[i],'Page':Page} for i in range(len(PosiList))}
                CordDictNow = {k:v for k,v in CordDictNow.items() if v}
                CordDict.update(CordDictNow)
                Page=Page+1

                for x in list(CordDictNow.keys()):
                  PositionListNow.remove(x)

                if len(PositionListNow)==0:
                    break
            dta[Year][Dept][Office]['Position']=CordDict

            ##Add Ending information
            Frame=pd.DataFrame.from_dict(dta[Year][Dept][Office]['Position'],orient='index')
            Frame["EndLocation"]=list(Frame['XLocation'].shift(-1))[:-1]+list([dta[Year][Dept][Office]["Office_X2"]])
            Frame["EndPage"]=list(Frame['Page'].shift(-1))[:-1]+list([dta[Year][NxDept][NxOffice]["Starting_Page"]])
            Frame=Frame.to_dict('index')
            dta[Year][Dept][Office]['Position']=Frame

Admin 職員課
{'Manager': 386, 'Leader': 463, 'Admin': 242}
{'Manager': 386, 'Leader': 463, 'Admin': 242}
{}
Admin 文書課
{'Manager': 473, 'Leader': 501, 'Admin': 256, 'Engineer2': 353}
{'Manager': 473, 'Leader': 501, 'Admin': 256, 'Engineer2': 353}
{}
監査局 庶務課
{'Leader': 173}
{'Leader': 173}
{'Manager': 190, 'Admin': 139}
監査局 監察課
{'Manager': 227, 'Leader': 254, 'Admin': 45}
{'Manager': 227, 'Leader': 254, 'Admin': 45}
{}
監査局 区政課
{'Manager': 361, 'Leader': 390, 'Admin': 213}
{'Manager': 361, 'Leader': 390, 'Admin': 213}
{}
監査局 統計課
監査局 都市計画課
財務局 主計課
{'Manager': 123, 'Leader': 124, 'Engineer2': 55}
{'Manager': 123, 'Leader': 124, 'Engineer2': 55}
{'Admin': 199}
財務局 公債課
{'Manager': 178, 'Leader': 212}
{'Manager': 178, 'Leader': 212}
{'Admin': 179}
財務局 収納課
{'Manager': 217, 'Leader': 242}
{'Manager': 217, 'Leader': 242}
{'Admin': 404, 'Engineer2': 382}
{'Manager': 217, 'Leader': 242}
{'Admin': 404, 'Engineer2': 382}
{}
{'Manager': 217, 'Leader': 242}
{'Admin': 404, 'Engineer2': 382}
{}
{}
{'Manager

IndexError: single positional indexer is out-of-bounds

In [68]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+str(Year)+'\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)

In [69]:
dta

{'1935': {'Admin': {'秘書課': {'Starting_Page': 2,
    'Office_X1': 416,
    'Ending_Page': 3,
    'Office_X2': 475,
    'Page_Range': [2, 3]},
   '職員課': {'Starting_Page': 3,
    'Office_X1': 465,
    'Ending_Page': 4,
    'Office_X2': 513,
    'Page_Range': [3, 4],
    'Position': {'Manager': {'XLocation': 386,
      'Page': 3,
      'EndLocation': 463.0,
      'EndPage': 3.0},
     'Leader': {'XLocation': 463,
      'Page': 3,
      'EndLocation': 242.0,
      'EndPage': 3.0},
     'Admin': {'XLocation': 242,
      'Page': 3,
      'EndLocation': 513.0,
      'EndPage': 4.0}}},
   '文書課': {'Starting_Page': 4,
    'Office_X1': 503,
    'Ending_Page': 5,
    'Office_X2': 185,
    'Page_Range': [4, 5],
    'Position': {'Manager': {'XLocation': 473,
      'Page': 4,
      'EndLocation': 501.0,
      'EndPage': 4.0},
     'Leader': {'XLocation': 501,
      'Page': 4,
      'EndLocation': 256.0,
      'EndPage': 4.0},
     'Admin': {'XLocation': 256,
      'Page': 4,
      'EndLocation': 353.0